In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1423


In [2]:
import LinearAlgebra as la

In [3]:
using BenchmarkTools: @btime

## HO Basis and Spatial System

In [536]:
l = 6
ω = 0.25
basis = SpinBasis(HOBasis(l, ω))

n = 2
V = ShieldedCoulomb(0.25)
grid = [x for x in range(-10, stop = 10, length = 2001)]
@time system = System(n, basis, grid, V);
l = 2 * l

  0.038586 seconds (455 allocations: 1.728 MiB)


12

## CCD

In [537]:
ccd = setup_CCD(system);

In [549]:
CCD_Update!(ccd);
energy(ccd)

0.8362892621367914

In [539]:
function pprint(tensor)
    display(round.(tensor[n+1:l, n+1:l, 1:n, 1:n], digits = 6)[:, :, 2, 1])
end

pprint (generic function with 1 method)

## CCSD

In [ ]:
ccsd = setup_CCSD(system, α = 0.5);

In [ ]:
for i in 1:10
    CCSD_Update!(ccsd)
end

In [ ]:
energy(ccsd)

In [ ]:
corr_energy(ccsd)

# HF

In [ ]:
hf = setup_HF(system);

In [ ]:
HF_update!(hf, iters = 10);

In [ ]:
energy(hf)

In [ ]:
system = System(hf);

## RHF

In [ ]:
rhf = setup_RHF(system);

In [ ]:
RHF_update!(rhf, iters = 10);

In [ ]:
energy(rhf)

In [ ]:
system = System(rhf);

In [387]:
mutable struct DIIS
    iter::Int
    
    max_vecs::Int
    
    trial_vecs::Vector{Vector{Float64}}
    direction_vecs::Vector{Vector{Float64}}
    error_vecs::Vector{Vector{Float64}}
end

function DIIS(shape, max_vecs = 10)
    vec_length = prod(shape)
    trial_vecs = [zeros(vec_length) for i in 1:max_vecs]
    direction_vecs = [zeros(vec_length) for i in 1:max_vecs]
    error_vecs = [zeros(vec_length) for i in 1:max_vecs]
    
    return DIIS(0, max_vecs, trial_vecs, direction_vecs, error_vecs)
end

function compute_new_vector(mixer::DIIS, trial, direction, error)
    (; iter, max_vecs, trial_vecs, direction_vecs, error_vecs) = mixer
    
    index = iter % max_vecs + 1
    iter += 1
    mixer.iter = iter
    
    error_vecs[index] .= vec(error)
    trial_vecs[index] .= vec(trial)
    direction_vecs[index] .= vec(direction)
    
    # Setting up the equations
    B_dim = min(iter, max_vecs) + 1
    B = zeros(B_dim, B_dim)
    for i in 1:B_dim-1
        for j in 1:i
            B[i, j] = la.dot(error_vecs[i], error_vecs[j])
            if i != j
                B[j, i] = B[i, j]
            end
        end
        B[i, B_dim] = -1
        B[B_dim, i] = -1
    end
    
    pre_condition = zeros(B_dim)
    if any(i <= 0 for i in la.diag(B)[1:end-1])
        pre_condition[1:end-1] .= 1
    else
        pre_condition[1:end-1] .= 1 ./ sqrt.(la.diag(B)[1:end-1])
    end
    pre_condition[end] = 1
    
    for i in 1:B_dim
        for j in 1:B_dim
            B[i, j] *= pre_condition[i] * pre_condition[j]
        end
    end
    # Solving the equations
    weights = -la.pinv(B)[end, :] # final row
    weights .*= pre_condition
    
    # Using solution to get new vector
    new_trial = zero(trial_vecs[1])
    for i in 1:B_dim-1
        new_trial .+= weights[i] .* (trial_vecs[i] .+ direction_vecs[i])
    end
    
    return reshape(new_trial, size(trial))
end

compute_new_vector (generic function with 1 method)